# Show how extinction curves work with sliders
<em> Just a hack! Don't trust this for your dust!

In [1]:
import numpy as np

from bokeh.io import output_file, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure, output_file, show, output_notebook

#output_file("slider.html")
output_notebook()


Loading BokehJS ...

In [2]:
# I did   conda install -c bokeh flexx    first


In [3]:
def getFMext(wave,R,source):
    
    """ 
        Input: wavelength in microns, nominal R = 3.1
            source='f99' for Fitzpatrick 1999 or ='fmunred' for fmunred.pro
            source='f99 tables' reproduces common misunderstanding from that paper
        Output: Al/EBV, so user must divide by R to get Al/AV !!!
    """
    
    from scipy.interpolate import interp1d
    
    import numpy as np

    x_anchors = 1.0E4 / np.array([np.inf, 26500., 12200., 6000., 5470., 4670., 4110.]) # microns
    
    if source=="f99 tables": # Don't use, for demonstration only
        a26500= 0.265
        a12200= 0.829
        a6000 = -0.426 +1.0044*R
        a5470 = -0.050 +1.0016*R
        a4670 =  0.701 +1.0016*R
        a4110 =  1.208 +1.0032*R -0.00033*R**2.0 # typo in the paper -KAL
    
    elif source=="f99": 
        a26500= 0.265*R/3.1
        a12200= 0.829*R/3.1
        a6000 = -0.426 +1.0044*R
        a5470 = -0.050 +1.0016*R
        a4670 =  0.701 +1.0016*R
        a4110 =  1.208 +1.0032*R -0.00033*R**2.0 # typo in the paper -KAL

    elif source=="fmunred":
        a26500= 0.26469*R/3.1
        a12200= 0.82925*R/3.1
        a6000 = -4.22809e-01 +1.00270*R+2.13572e-04*R**2.0
        a5470 = -5.13540e-02 +1.00216*R-7.35778e-05*R**2.0
        a4670 =  7.00127e-01 +1.00184*R-3.32598e-05*R**2.0
        a4110 =  1.19456 +1.01707*R -5.46959e-03*R**2.0+ 7.97809e-04*R**3.0 -4.45636e-05*R**4.0

    a_anchors = np.array([0.0, a26500, a12200, a6000, a5470, a4670, a4110])

    f=interp1d(x_anchors, a_anchors, kind='cubic')    
    
    
    return f(1.0/wave)

In [4]:
x=np.arange(0.5,5,.01)
invx=1./x

y21=getFMext(x,2.1,"f99 tables")/2.1
y31=getFMext(x,3.1,"f99 tables")/3.1
y41=getFMext(x,4.1,"f99 tables")/4.1
y51=getFMext(x,5.1,"f99 tables")/5.1

y=y31


In [5]:
source = ColumnDataSource(data=dict(x=x, invx=invx, y=y, y21=y21, y31=y31, y41=y41, y51=y51))

plot1 = Figure(width=300, height=300, x_range=[0,5], y_range=[0,2], x_axis_label='lambda (microns)', y_axis_label='A(lambda)', toolbar_location=None)
plot1.line('x', 'y', source=source, line_width=3, line_color='red')

plot2 = Figure(width=300, height=300, x_range=[0,2], y_range=[0,2], x_axis_label='1/lambda (1/microns)', y_axis_label='A(lambda)', toolbar_location=None)
plot2.line('invx', 'y', source=source, line_width=3, line_color='red')

slider1 = Slider(start=1., end=5., value=1., step=.1, title="A(V)")
slider2 = Slider(start=2.1, end=5.1, value=3.1, step=1., title="R(V)")

def update(source=source, slider1=slider1, slider2=slider2, window=None):
    data = source.data
    y = data['y']
    y1 = data['y31']
    if slider2.value==2.1:
        y1 = data['y21']
    if slider2.value==4.1:
        y1 = data['y41']
    if slider2.value==5.1:
        y1 = data['y51']
    for i in range(len(y)):
        y[i] = slider1.value*y1[i]
    source.trigger('change')


slider1.js_on_change('value', CustomJS.from_py_func(update))
slider2.js_on_change('value', CustomJS.from_py_func(update))

show(column(slider1, slider2, row(plot1, plot2)))


The take-home message is that <code>CustomJS.from_py_func</code> will convert some (but not all) python code into java on update of the slider.  Notably, numpy doesn't work and function calls are not allowed.

The next step would be synthetic photometry, but I would have to pre-calculate colors for all the models ahead of the sliders.  Is it worth the trouble just for the demonstration?  We'll see. 